In [1]:
import requests
import json
import time
import psycopg2
import numpy as np
from openeye.oechem import *
import progressbar


def molFromString(mol_string):
    oemol = OEGraphMol()
    ifs = oemolistream()
    ifs.SetFormat(OEFormat_SDF)
    ifs.openstring(mol_string)
    OEReadMolecule(ifs,oemol)
    ifs.close()
    return oemol

class SkyhawkDb:
    def __init__(self):
        self.conn = psycopg2.connect(database='skyhawk',user='medchem',host='10.74.2.128',password='medchem')
        
    def get_molecule(self,id):
        cursor = None
        try:
            cursor = self.conn.cursor()
            cursor.execute("select molfile,corporate_id from chemicals where chemical_id = %s",(id,))
            row = cursor.fetchone()
            if row is not None:
                return row[0],row[1]
            return None
        finally:
            if cursor is not None:
                cursor.close()
                
    def close(self):
        self.conn.close()
        
    def __del__(self):
        self.conn.close()
        

class ReadoutDef:
    def __init__(self, jsonText):
        dict = json.loads(jsonText)
        self.name = dict['name']
        self.data_type = dict['data_type']
        self.unit_label = dict['nM']
        self.id = dict['id']
    def __str__(self):
        return "%d %s %s %s"%(self.id,self.name,self.data_type,self.unit_label)

adme_protocol_names = [
    "General Metabolic Stability",
    "General hERG",
    "General MDCK-MDR1"
]
       
keywords = ["data_sets","molecules","projects","protocols"]
#api_key = "NjE3fHpPaEVEM09KZUpsWFFtbTE4Z01RdXY3STVQRXFGTjhXUTdBQzNEY210blpyRG96cHFRPT0=" #sandbox
api_key = "NjE4fHJSTWt2NjQxZ2pHY0ZKSDlkZFRGeUIwc2h4Mm8zTERoZ0xKbXVOZVFSOWJXVVF6Q09RPT0=" #production readonly
cdd_url = "https://app.collaborativedrug.com/api/v1/vaults/4647"
headers = {'X-CDD-token':api_key}

def is_async_export_finished(id):
    dict = json.loads(requests.request("GET","%s/export_progress/%d"%(cdd_url,id),headers=headers).text)
    print(dict['status'])
    if dict['status']=="finished":
        return True
    else:
        return False

def get_cdd_protocols():
    dict = json.loads(requests.request("GET","%s/protocols/"%(cdd_url), headers=headers).text)
    return dict['objects']

def get_protocol_data(id):
    max_time = 100
    dict = json.loads(requests.request("GET","%s/protocols/%d/data?async=true"%(cdd_url,id),headers=headers).text)
    export_id = dict['id']
    success = False
    start_time = time.time()
    pgbar = progressbar.ProgressBar(0,100)
    while True:
        time.sleep(1)
        elapsed_time = time.time()-start_time+1
        pgbar.update(int(elapsed_time))
        if elapsed_time>max_time:
            break
        if is_async_export_finished(export_id):
            success = True
            break
    if success:
        return (requests.request("GET","%s/exports/%d"%(cdd_url,export_id),headers=headers)).text
    else:
        return None


In [2]:

protocols = get_cdd_protocols()
protocol = None
adme_protocols = []

for object in protocols:
    if object['name'] == 'HTT Potency':
        protocol = object
    elif object['name'] in adme_protocol_names:
        adme_protocols.append(object)
        
readout_definitions = protocol['readout_definitions']
protocol_data = get_protocol_data(int(protocol['id']))
adme_data_dict = {}
adme_definitions_dict = {}
for adme_protocol in adme_protocols:
    protocol_name = adme_protocol['name']
    tmp_data = get_protocol_data(adme_protocol['id'])
    adme_data_dict[protocol_name] = tmp_data
    adme_definitions_dict[protocol_name] = adme_protocol['readout_definitions']
print(adme_definitions_dict)
print(adme_data_dict)
    



N/A% (0 of 100) |                        | Elapsed Time: 0:00:00 ETA:  --:--:--

started


  3% (3 of 100) |                        | Elapsed Time: 0:00:01 ETA:   0:00:44

started


  4% (4 of 100) |                        | Elapsed Time: 0:00:02 ETA:   0:01:06

...

  3% (3 of 100) |                        | Elapsed Time: 0:00:01 ETA:   0:00:44

started


  4% (4 of 100) |                        | Elapsed Time: 0:00:02 ETA:   0:01:05

finished


{'General MDCK-MDR1': [{'name': '+ P-gp Inhibitor', 'created_at': '2018-10-26T13:57:56.000Z', 'data_type': 'PickList', 'protocol_condition': True, 'modified_at': '2018-10-30T12:41:10.000Z', 'class': 'readout definition', 'id': 418460, 'description': 'Whether or not a P-gp inhibitor (Valspodar) was used in the assay'}, {'name': '% Recov A-B', 'created_at': '2018-10-26T13:58:33.000Z', 'data_type': 'Number', 'protocol_condition': False, 'modified_at': '2018-10-26T13:58:33.000Z', 'unit_label': '%', 'class': 'readout definition', 'precision_type': 'decimal place', 'id': 418461, 'precision_number': 0}, {'name': '% Recov B-A', 'created_at': '2018-10-26T13:59:08.000Z', 'data_type': 'Number', 'protocol_condition': False, 'modified_at': '2018-10-26T13:59:08.000Z', 'unit_label': '%', 'class': 'readout definition', 'precision_type': 'decimal place', 'id': 418462, 'precision_number': 0}, {'name': 'Papp A-B', 'created_at': '2018-10-26T13:59:30.000Z', 'data_type': 'Number', 'protocol_condition': Fals

In [3]:

#{'molecule': 53706260, 'readouts': {'413437': {'modifier': '>', 'note': '(n=4)', 'value': 10000.0}, '414205': {'modifier': '<=>', 'value': 1.0}, '413440': {'modifier': '<=>', 'value': 1.0}, '412001': {'modifier': '<=>', 'value': 1.0}, '412003': {'modifier': '>', 'value': 3784.4}, '411999': {'modifier': '>', 'value': 10000.0}, '413438': {'modifier': '>', 'note': '(n=4)', 'value': 10000.0}, '412000': {'modifier': '>', 'value': 10000.0}}, 'batch': 51018110, 'id': 264405693, 'run': 235896}
def_dict = {}
for definition in readout_definitions:
    def_dict[str(definition['id'])] = definition
for adme_name in adme_protocol_names:
    definitions = adme_definitions_dict[adme_name]
    for definition in definitions:
        def_dict[str(definition['id'])] = definition
print(def_dict)
    

# readout_data = json.loads(protocol_data.text)
# print(readout_data.keys())
# mol_data = readout_data['objects']
# for id,mol_dat in enumerate(mol_data):
#     print(id,mol_dat)
    # print (mol_dat['molecule'])
    # print(json.loads(get_molecule(mol_dat['molecule']).text).keys())

{'418465': {'name': 'Efflux', 'created_at': '2018-10-26T13:59:53.000Z', 'data_type': 'Number', 'protocol_condition': False, 'modified_at': '2018-10-26T13:59:53.000Z', 'class': 'readout definition', 'precision_type': 'decimal place', 'id': 418465, 'precision_number': 0}, '412000': {'description': 'cytotoxicity', 'name': 'CC50 CTG1', 'created_at': '2018-09-28T13:42:55.000Z', 'data_type': 'Number', 'protocol_condition': False, 'modified_at': '2018-10-03T18:06:45.000Z', 'unit_label': 'nM', 'class': 'readout definition', 'precision_type': 'decimal place', 'id': 412000, 'precision_number': 1}, '418466': {'name': 'MDCK-MDR1.pdf', 'created_at': '2018-10-26T14:00:04.000Z', 'data_type': 'File', 'protocol_condition': False, 'modified_at': '2018-10-26T14:00:04.000Z', 'class': 'readout definition', 'id': 418466}, '412002': {'name': 'Protein and Cytotox.xlsx', 'created_at': '2018-09-28T13:52:59.000Z', 'data_type': 'File', 'protocol_condition': False, 'modified_at': '2018-10-25T15:54:06.000Z', 'class

In [4]:
data_dict = json.loads(protocol_data)
print(data_dict.keys())

dict_keys(['count', 'objects'])


In [5]:
mol_dict = {}
mol_id_list = []
i = 0
mol_data_dict = {}
mol_count = data_dict['count']
molecules = data_dict['objects']
for molecule in molecules:
    i = i+1
    mol_id = molecule['molecule']
    mol_id_list.append(mol_id)
    if mol_id not in mol_data_dict:
        mol_data_dict[mol_id] = {}


    for key in molecule['readouts'].keys():
        definition = def_dict[key]
        def_name = definition['name']
        def_type = definition['data_type']
        if def_type == 'Number':
            data_point = molecule['readouts'][key]
            print(data_point)
            if def_name not in mol_data_dict[mol_id]:
                mol_data_dict[mol_id][def_name] = []
            if isinstance(data_point,(int,float)):
                mol_data_dict[mol_id][def_name].append(data_point)
            elif 'value' in data_point:
                mol_data_dict[mol_id][def_name].append(data_point['value'])
            else:
                continue
            
    for adme in adme_protocol_names:
        adme_data = json.loads(adme_data_dict[adme])
        for mol in adme_data['objects']:
            if mol['molecule']==mol_id:
                for key in mol['readouts'].keys():
                    definition = def_dict[key]
                    def_name = definition['name']
                    def_type = definition['data_type']
                    if def_type == 'Number':
                        data_point = mol['readouts'][key]
                        if def_name not in mol_data_dict[mol_id]:
                            mol_data_dict[mol_id][def_name] = []
                        if isinstance(data_point,(int,float)):
                            mol_data_dict[mol_id][def_name].append(data_point)
                        elif 'value' in data_point:
                            mol_data_dict[mol_id][def_name].append(data_point['value'])
                        else:
                            continue
        
print(mol_data_dict)

    # print(oemol.NumAtoms())

{'value': 3784.4, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'value': 10000.0, 'modifier': '>'}
{'value': 1.0, 'modifier': '<=>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 3784.4, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'value': 10000.0, 'modifier': '>'}
{'value': 1.0, 'modifier': '<=>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 3784.4, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.

{'value': 3764.64, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'value': 10000.0, 'modifier': '>'}
{'value': 1.0, 'modifier': '<=>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 3764.64, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'value': 10000.0, 'modifier': '>'}
{'value': 1.0, 'modifier': '<=>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 3764.64, 'modifier': '>'}
{'note': '(n=4)', 'value': 10000.0, 'modifier': '>'}
{'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitio

{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
651.8
{'value': 19.6966712625566, 'modifier': '>'}
{'note': '(n=4)', 'value': 495.015}
{'value': 3774.48, 'modifier': '>'}
{'note': '(n=2)', 'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
388.97
{'value': 19.6966712625566, 'modifier': '>'}
{'note': '(n=4)', 'value': 495.015}
{'value': 3774.48, 'modifier': '>'}
{'note': '(n=2)', 'value': 10000.0, 'modifier': '>'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
513.26
{'value': 19.6966712625566, 'modifier': '>'}
{'note': '(n=4)', 'value': 495.015}
97.504007425
{'note': '(n=4)', 'value': 262.515}
267.36
{'note': 'could not be calculated. Missin

{'note': '(n=4)', 'value': 197.725}
229.64
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
4.84
39.0303178484108
{'note': '(n=4)', 'value': 4.855}
96.08929179
{'note': '(n=4)', 'value': 197.725}
193.81
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
4.87
39.0303178484108
{'note': '(n=4)', 'value': 4.855}
96.08929179
{'note': '(n=4)', 'value': 197.725}
201.64
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
4.84
39.0303178484108
{'note': '(n=4)', 'value': 4.855}
1315.18520524
{'note': '(n=4)', 'value': 3007.59}
3261.58
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
47.0


2337.557533885
{'note': '(n=4)', 'value': 5849.825}
5022.34
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
50.2
114.069580162043
{'note': '(n=4)', 'value': 48.975}
2337.557533885
{'note': '(n=4)', 'value': 5849.825}
6395.92
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
47.75
114.069580162043
{'note': '(n=4)', 'value': 48.975}
2337.557533885
{'note': '(n=4)', 'value': 5849.825}
6508.28
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
34.62
114.069580162043
{'note': '(n=4)', 'value': 48.975}
3551.119152555
{'note': '(n=4)', 'value': 7961.28}
7151.47
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing refere

...

{'note': '(n=1)', 'value': 1000.0, 'modifier': '>'}
{'value': 5815.44, 'modifier': '>'}
{'note': '(n=2)', 'value': 16000.0, 'modifier': '>'}
{'value': 16000.0, 'modifier': '>'}
{'value': 250.0, 'modifier': '>'}
{'value': 16.0, 'modifier': '<=>'}
{'note': '(n=1)', 'value': 1000.0, 'modifier': '>'}
207.526815276
{'note': '(n=4)', 'value': 590.4398}
417.4888
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
1.3345
229.911372475419
{'note': '(n=4)', 'value': 2.2962}
207.526815276
{'note': '(n=4)', 'value': 590.4398}
575.3953
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
1.104
229.911372475419
{'note': '(n=4)', 'value': 2.2962}
{'value': 5815.44, 'modifier': '>'}
{'note': '(n=2)', 'value': 16000.0, 'modifier': '>'}
{'value': 16000.0, 'modifier': '>'}
{'value': 1000.0, 'modifier': '>


9.3511
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
0.335
16.8290873911641
{'note': '(n=2)', 'value': 0.46515}
351.5885012112
{'note': '(n=2)', 'value': 933.9233}
826.3358
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
3.5352
313.255169637916
{'note': '(n=2)', 'value': 2.98135}
351.5885012112
{'note': '(n=2)', 'value': 933.9233}
1041.5108
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
2.4275
313.255169637916
{'note': '(n=2)', 'value': 2.98135}
3359.10282916725
{'note': '(n=2)', 'value': 11001.15225}
7472.0354
{'value': 1000.0, 'modifier': '>'}
{'value': 11.00115225, 'modifier': '<'}
{'note': '(n=2)', 'value': 1000.0, 'modifier': '>'}
3359.10282916725
{'note': '(n=2)',

{'note': '(n=2)', 'value': 166.2573}
2362.3537285496
{'note': '(n=2)', 'value': 7282.4942}
8263.6859
159.8337
43.8025530307541
{'note': '(n=2)', 'value': 166.2573}
94.3120447572
{'note': '(n=2)', 'value': 249.86765}
217.5867
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
0.1455
1421.72204836415
{'note': '(n=2)', 'value': 0.17575}
94.3120447572
{'note': '(n=2)', 'value': 249.86765}
282.1486
{'note': 'could not be calculated. Missing referenced readout definitions.'}
{'note': 'could not be calculated. Missing referenced readout definitions.'}
0.206
1421.72204836415
{'note': '(n=2)', 'value': 0.17575}
{'value': 5027.407079541, 'modifier': '>'}
{'note': '(n=2)', 'value': 14855.7454, 'modifier': '>'}
13711.4908
151.054
{'value': 126.02158165055, 'modifier': '>'}
{'note': '(n=2)', 'value': 117.88255}
{'value': 5027.407079541, 'modifier': '>'}
{'note': '(n=2)', 'value': 14855.7454, 'modi


{'note': '(n=2)', 'value': 995.6807}
973.5126
5.709
141.911675835923
{'note': '(n=2)', 'value': 7.0162}
309.0204577327
{'note': '(n=2)', 'value': 995.6807}
1017.8488
8.3234
141.911675835923
{'note': '(n=2)', 'value': 7.0162}
{'value': 5190.208, 'modifier': '>'}
{'note': '(n=2)', 'value': 16000.0, 'modifier': '>'}
{'value': 16000.0, 'modifier': '>'}
{'value': 1000.0, 'modifier': '>'}
{'value': 16.0, 'modifier': '<=>'}
{'note': '(n=2)', 'value': 1000.0, 'modifier': '>'}
{'value': 5190.208, 'modifier': '>'}
{'note': '(n=2)', 'value': 16000.0, 'modifier': '>'}
{'value': 16000.0, 'modifier': '>'}
{'value': 1000.0, 'modifier': '>'}
{'value': 16.0, 'modifier': '<=>'}
{'note': '(n=2)', 'value': 1000.0, 'modifier': '>'}
{'value': 6039.232, 'modifier': '>'}
{'note': '(n=2)', 'value': 16000.0, 'modifier': '>'}
{'value': 16000.0, 'modifier': '>'}
{'value': 1000.0, 'modifier': '>'}
{'value': 16.0, 'modifier': '<=>'}
{'note': '(n=2)', 'value': 1000.0, 'modifier': '>'}
{'value': 6039.232, 'modifier'

15336.6107
{'value': 1000.0, 'modifier': '>'}
{'value': 14.74793185, 'modifier': '<'}
{'note': '(n=2)', 'value': 1000.0, 'modifier': '>'}
3006.326473337
{'note': '(n=2)', 'value': 6887.551}
5596.7516
9.2252
640.854435238126
{'note': '(n=2)', 'value': 10.74745}
3006.326473337
{'note': '(n=2)', 'value': 6887.551}
8178.3504
12.2697
640.854435238126
{'note': '(n=2)', 'value': 10.74745}
2648.4659717846
{'note': '(n=2)', 'value': 6067.6858}
5459.0312
22.2746
266.879216386564
{'note': '(n=2)', 'value': 22.7357}
2648.4659717846
{'note': '(n=2)', 'value': 6067.6858}
6676.3404
23.1968
266.879216386564
{'note': '(n=2)', 'value': 22.7357}
1796.29680962925
{'note': '(n=2)', 'value': 4355.02845}
4593.0181
12.4069
304.906722256645
{'note': '(n=2)', 'value': 14.28315}
1796.29680962925
{'note': '(n=2)', 'value': 4355.02845}
4117.0388
16.1594
304.906722256645
{'note': '(n=2)', 'value': 14.28315}
1621.55696104875
{'note': '(n=2)', 'value': 3931.38075}
3510.831
22.1118
196.438896730916
{'note': '(n=2)', '

In [7]:
molDb = SkyhawkDb()
ofs = oemolostream()
ofs.open("/home/jfeng/cdd_export.sdf")
for mol_id in mol_data_dict.keys():
    molfile,mol_name = molDb.get_molecule(mol_id)
    oemol = molFromString(molfile)
    oemol.SetTitle(mol_name)
    OESetSDData(oemol,"mol_id",str(mol_id))
    for tag in mol_data_dict[mol_id].keys():
        try:
            values = np.array(mol_data_dict[mol_id][tag])
            avg_value = np.mean(values)
            OESetSDData(oemol,tag,"%5.3f"%avg_value)
        except:
            OESetSDData(oemol,tag,"")
    OEWriteMolecule(ofs,oemol)
ofs.close()




/home/jfeng/anaconda3/envs/compchem/lib/python3.5/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jfeng/anaconda3/envs/compchem/lib/python3.5/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
